# Example: scenarios for TAMU or RTS grids


In [1]:
import pandas as pd
import numpy as np
import time
import os

# # set PYWTK_CACHE_DIR to locate WIND Toolkit data
# # will download from AWS as needed
# os.environ["PYWTK_CACHE_DIR"] = os.path.join(os.environ["HOME"], "pywtk-data")

from powerscenarios.parser import Parser
#from powerscenarios.grid import Grid
from powerscenarios.grid_copy import Grid

# show multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 100)

# plotting (optional)
import cufflinks as cl

cl.go_offline()


<IPython.core.display.Javascript object>

In [2]:
Grid.blet

'blet'

<IPython.core.display.Javascript object>

# choose grid

# parse TAMU grid .aux files

In [3]:
# choose TAMU grid
# grid files can be downloaded from:
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/

grid_name = "ACTIVSg200"  # TAMU 200 bus case
# grid_name = "ACTIVSg2000"  # TAMU 2000 bus case
# grid_name = 'ACTIVSg10k' # TAMU 10000 bus case

# path to .aux file (TAMU grids) obtained from e.g.
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/activsg200/
data_dir = "../data/grid-data/"
# aux_file_name = data_dir + grid_name + "/" + grid_name + ".aux"
aux_file_name = os.path.join(data_dir, grid_name, grid_name + ".aux")

# parse original .aux file and return dataframes for buses, generators, and wind generators
# here, we need .aux files because those are the only ones with Latitute/Longitude information
parser = Parser()
bus_df, gen_df, wind_gen_df = parser.parse_tamu_aux(aux_file_name)

# see what you got
print("bus_df:")
bus_df.head()
print("gen_df:")
gen_df.head()
print("wind_gen_df:")
wind_gen_df

bus_df:


,BusNum,BusName,Latitude,Longitude,Zone
0,1,CREVECOEUR0,40.642116,-89.599560,2
1,2,CREVECOEUR1,40.642116,-89.599560,2
2,3,ILLIOPOLIS0,39.866030,-89.251291,4
3,4,ILLIOPOLIS1,39.866030,-89.251291,4
4,5,PAXTON20,40.378337,-88.105151,6


gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,49,1,4.53,1.36,1.0,Coal,49_Coal_1,RANTOUL21,40.312222,-88.159444,6
1,50,1,4.53,1.36,1.0,Coal,50_Coal_1,RANTOUL22,40.312222,-88.159444,6
2,51,1,4.53,1.36,1.0,Coal,51_Coal_1,RANTOUL23,40.312222,-88.159444,6
3,52,1,4.53,1.36,1.0,Coal,52_Coal_1,RANTOUL24,40.312222,-88.159444,6
4,53,1,9.07,2.72,1.0,Coal,53_Coal_1,RANTOUL25,40.312222,-88.159444,6


wind_gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,65,1,150.399995,45.120001,1.0,Wind,65_Wind_1,PAXTON11,40.46405,-88.021517,6
1,104,1,99.000001,29.699999,1.0,Wind,104_Wind_1,ELLSWORTH12,40.47920,-88.798900,7
2,105,1,198.000002,59.400004,1.0,Wind,105_Wind_1,ELLSWORTH13,40.47920,-88.798900,7
3,114,1,1.700000,0.510000,1.0,Wind,114_Wind_1,NORMAL22,40.53700,-89.019000,7
4,115,1,150.000000,44.999999,1.0,Wind,115_Wind_1,NORMAL23,40.53700,-89.019000,7
5,147,1,100.500000,30.149999,1.0,Wind,147_Wind_1,HOPEDALE21,40.36920,-89.402200,7


<IPython.core.display.Javascript object>

# or parse RTS grid .csv files

In [4]:
# # RTS-GMLC grid
# # https://github.com/GridMod/RTS-GMLC
# grid_name = "RTS"

# data_dir = "../data/grid-data"

# bus_csv_filename = os.path.join(data_dir, grid_name, "bus.csv")
# gen_csv_filename = os.path.join(data_dir, grid_name, "gen.csv")

# parser = Parser()

# # if solar2wind, will replace all solar to wind
# bus_df, gen_df, wind_gen_df = parser.parse_rts_csvs(
#     bus_csv_filename, gen_csv_filename, solar2wind=False
# )

# # see what you got
# print("bus_df:")
# bus_df.head()
# print("gen_df:")
# gen_df.head()
# print("wind_gen_df:")
# wind_gen_df.head()

<IPython.core.display.Javascript object>

# Grid etc.
* instantiate grid
* retrieve wind sites
* make tables

In [5]:
# to instantiate a grid we need: name, bus, generator, and wind generator dataframes from Parser
# really, we only wind generators, will change in the future
grid = Grid(grid_name, bus_df, gen_df, wind_gen_df)
grid
print(grid.info())

# retrieve wind sites matching current wind penetration

# retrieve wind sites (wind_sites are initially set to empty df )
grid
grid.retrieve_wind_sites(method="simple proximity")
grid
grid.wind_sites.head()


grid.make_tables(
    actuals_start=pd.Timestamp("2007-01-01 00:00:00", tz="utc"),
    actuals_end=pd.Timestamp("2007-12-31 23:55:00", tz="utc"),
    scenarios_start=pd.Timestamp("2008-01-01 00:00:00", tz="utc"),
    scenarios_end=pd.Timestamp("2013-12-31 23:55:00", tz="utc"),
)

# for actuals, make year you want
grid.actuals.index = grid.actuals.index.map(lambda t: t.replace(year=2020))
# see what you got
print("\nactuals_df:")
grid.actuals.head()
print("\nscenarios_df:")
grid.scenarios.head()



Grid(name=ACTIVSg200, buses=200, generators=49, wind_generators=6, wind_sites=0)


ACTIVSg200 grid info: 

 number of buses: 200
 number of generators: 49
 number of wind generators: 6
 number of solar generators: 0
 total generator capacity: 3602.84 MW
 wind capacity/penetration: 699.60 MW / 19.42%
 solar capacity/penetration: 0.00 MW / 0.00%


Grid(name=ACTIVSg200, buses=200, generators=49, wind_generators=6, wind_sites=0)

Retrieving wind sites ...
Done


Grid(name=ACTIVSg200, buses=200, generators=49, wind_generators=6, wind_sites=50)

...

Retrieving WTK data ...
Done
Retrieving WTK data ...
Done

actuals_df:


,65_Wind_1,104_Wind_1,105_Wind_1,114_Wind_1,115_Wind_1,147_Wind_1,TotalPower
IssueTime,,,,,,,
2020-01-01 00:00:00+00:00,142.094736,99.000001,198.000002,1.7,150.0,100.5,691.294739
2020-01-01 00:05:00+00:00,140.804726,99.000001,198.000002,1.7,150.0,100.5,690.004728
2020-01-01 00:10:00+00:00,142.361249,99.000001,198.000002,1.7,150.0,100.5,691.561251
2020-01-01 00:15:00+00:00,145.972502,99.000001,198.000002,1.7,150.0,100.5,695.172504
2020-01-01 00:20:00+00:00,150.399995,99.000001,198.000002,1.7,150.0,100.5,699.599997



scenarios_df:


,65_Wind_1,104_Wind_1,105_Wind_1,114_Wind_1,115_Wind_1,147_Wind_1,TotalPower,Deviation
IssueTime,,,,,,,,
2008-01-01 00:00:00+00:00,1.459344,0.668133,0.583035,-0.024500,0.877288,1.845110,96.746560,5.408411
2008-01-01 00:05:00+00:00,1.601367,-0.445220,1.269019,0.091567,1.525665,3.528319,102.154971,7.570718
2008-01-01 00:10:00+00:00,1.097983,-0.016742,1.678258,0.294259,3.257260,5.579959,109.725689,11.890977
2008-01-01 00:15:00+00:00,1.172363,2.034480,3.025435,0.001593,4.814818,3.355411,121.616666,14.404100
2008-01-01 00:20:00+00:00,1.237113,2.234493,2.933309,0.000000,5.278723,6.783124,136.020766,18.466761


<IPython.core.display.Javascript object>

## generate_wind_scenarios

In [6]:
t0 = time.time()
# time period for which to generate scenarios

# a few timestamps timestamp
sim_timestamps = [
    pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),
]
# sim_timestamps = [pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),pd.Timestamp("2020-07-01 00:20:00+0000", tz="UTC")]

# #range
# sim_timestamps = pd.date_range(
#    start=pd.Timestamp("2020-07-01 00:00:00+0000", tz="UTC"), end=pd.Timestamp("2020-07-07 00:00:00+0000", tz="UTC"), freq="5min"
# )


# other parameters
# sampling_method="monte carlo"
sampling_method = "importance"
fidelity = "checkmark"
n_scenarios = 3
n_periods = 2

########################################################

all_weights_df = pd.DataFrame(index=sim_timestamps, columns=range(1, n_scenarios + 1))

# create multiindex df for all generated scenarios
# three arrays for multiindex:
a1 = [x for x in sim_timestamps for k in range(n_scenarios * n_periods)]
a2 = [x for x in range(1, n_scenarios + 1) for k in range(n_periods)] * len(
    sim_timestamps
)
a3 = [
    t + pd.Timedelta("5min") * k
    for t in sim_timestamps
    for k in list(range(n_periods)) * n_scenarios
]

index = pd.MultiIndex.from_arrays(
    [a1, a2, a3], names=["sim_timestamp", "scenario_nr", "period_timestamp"]
)
all_scenarios_df = pd.DataFrame(
    index=index, columns=grid.wind_generators["GenUID"].values
)


for sim_timestamp in sim_timestamps:
    # print("sim_timestamp = {}".format(sim_timestamp))
    random_seed = np.random.randint(2 ** 31 - 1)
    # random_seed = 594081473
    # print("random_seed = {}".format(random_seed))
    scenarios_df, weights_df = grid.generate_wind_scenarios(
        sim_timestamp,
        power_quantiles=[0.0, 0.1, 0.9, 1.0],
        sampling_method=sampling_method,
        fidelity=fidelity,
        n_scenarios=n_scenarios,
        n_periods=n_periods,
        # random_seed=6,
        random_seed=random_seed,
        output_format=0,
    )
    # all_scenarios_df=pd.concat([all_scenarios_df,scenarios_df])
    all_scenarios_df.loc[sim_timestamp] = scenarios_df

    # all_weights_df=pd.concat([all_weights_df,weights_df])
    all_weights_df.loc[sim_timestamp] = weights_df.loc[sim_timestamp]


# copy wanted actuals
all_actuals_df = grid.actuals.loc[sim_timestamps].drop("TotalPower", axis=1).copy()
# match index name to all_scenarios_df
all_actuals_df.index.name = "sim_timestamp"

print("\nall_actuals_df:")
all_actuals_df
print("\nall_scenarios_df:")
all_scenarios_df
print("\nall_weights_df:")
all_weights_df

t1 = time.time()

print("elapsed time = {}".format(t1 - t0))


all_actuals_df:


,65_Wind_1,104_Wind_1,105_Wind_1,114_Wind_1,115_Wind_1,147_Wind_1
sim_timestamp,,,,,,
2020-07-01 00:15:00+00:00,31.596573,9.109061,27.28363,1.201169,17.069507,14.975642



all_scenarios_df:


65_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                      
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00   31.4618   
                                      2020-07-01 00:20:00+00:00   31.3621   
                          2           2020-07-01 00:15:00+00:00   31.5484   
                                      2020-07-01 00:20:00+00:00   31.5596   
                          3           2020-07-01 00:15:00+00:00   29.2674   
                                      2020-07-01 00:20:00+00:00   26.4687   

                                                                104_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00    9.97715   
                                      2020-07-01 00:20:00+00:00    9.79946   
                          2           2020-07-01 00:15:00+00:00    9.27446   
                                      2020-07-01 00:20:00+00:00    8.88956   
                          3           2020-07-01 00:15:00+00:00     2.8513   
                                      2020-07-01 00:20:00+00:00    28.8226   

                                                                105_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00    25.6717   
                                      2020-07-01 00:20:00+00:00    25.2903   
                          2           2020-07-01 00:15:00+00:00    24.4801   
                                      2020-07-01 00:20:00+00:00    23.2904   
                          3           2020-07-01 00:15:00+00:00    26.0274   
                                      2020-07-01 00:20:00+00:00    52.7995   

                                                                114_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00    1.18629   
                                      2020-07-01 00:20:00+00:00    1.12741   
                          2           2020-07-01 00:15:00+00:00    1.17654   
                                      2020-07-01 00:20:00+00:00     1.1403   
                          3           2020-07-01 00:15:00+00:00    1.22973   
                                      2020-07-01 00:20:00+00:00    1.22973   

                                                                115_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00    14.3289   
                                      2020-07-01 00:20:00+00:00    13.5194   
                          2           2020-07-01 00:15:00+00:00    14.8934   
                                      2020-07-01 00:20:00+00:00    15.1128   
                          3           2020-07-01 00:15:00+00:00    3.59428   
                                      2020-07-01 00:20:00+00:00          0   

                                                                147_Wind_1  
sim_timestamp             scenario_nr period_timestamp                      
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00    11.8343  
                                      2020-07-01 00:20:00+00:00    11.3342  
                          2           2020-07-01 00:15:00+00:00     12.344  
                                      2020-07-01 00:20:00+00:00    11.9208  
                          3           2020-07-01 00:15:00+00:00    12.9837  
                                      2020-07-01 00:20:00+00:00    12.9837


all_weights_df:


,1,2,3
2020-07-01 00:15:00+00:00,0.18954,0.189145,0.712887


elapsed time = 189.04999780654907


<IPython.core.display.Javascript object>

# plot 

In [7]:
# # total power
# # choose sim_timestamp for which to plot all scenarios
# sim_timestamp = sim_timestamps[0]

# # all needed period timestamps: t0,t1,...
# timestamps = pd.date_range(
#     start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
# )
# plot_df = pd.DataFrame(index=timestamps, columns=range(1, n_scenarios + 1),)
# for scenario_nr in range(1, n_scenarios + 1):
#     s = all_scenarios_df.loc[(sim_timestamp, scenario_nr,)].sum(axis=1)
#     s.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]].loc["TotalPower"]

#     plot_df[scenario_nr] = s

# #plot_df.iplot()
# #plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power", asImage=True,)
# plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power")




<IPython.core.display.Javascript object>

In [8]:
# # power at generators
# # choose sim_timestamp and scenario nr for which to plot power at generators
# sim_timestamp = sim_timestamps[0]
# scenario_nr = 1

# # all needed period timestamps: t0,t1,...
# timestamps = pd.date_range(
#     start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
# )

# plot_df = all_scenarios_df.loc[(sim_timestamp, 1)]
# plot_df.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]]
# plot_df.sort_index(inplace=True)

# # plot_df.iplot()
# # plot_df.iplot(xTitle="Time", yTitle="MW", title="scenario_nr={}".format(scenario_nr), asImage=True,)
# plot_df.iplot(
#     xTitle="Time",
#     yTitle="MW",
#     title="Power at generators when scenario_nr={}".format(scenario_nr),
# )

<IPython.core.display.Javascript object>

# Importance sampling weights

$E_{f}\left[y\left(X\right)\right]=\int y\left(x\right)f(x)dx=\int y\left(x\right)\frac{f(x)}{g(x)}g(x)dx=E_{g}\left[y\left(X\right)\frac{f(X)}{g(X)}\right]$


$E\left[y\left(X\right)\right]\approx\frac{1}{N_{s}}{\displaystyle \sum_{i=1}^{N_{s}}y\left(X_{i}\right)} \;\;\; \textrm{with} \;\; X_i \; \textrm{from} \;\;f$


$E\left[y\left(X\right)\frac{f(X)}{g(X)}\right]\approx\frac{1}{N_{s}}{\displaystyle \sum_{i=1}^{N_{s}}y\left(X_{i}\right)\frac{f(X_i)}{g(X_i)}} \;\;\; \textrm{with} \;\; X_i \; \textrm{from} \;\;g$

$\frac{f(X_i)}{g(X_i)} = w_i \;\; \textrm{are importance sampling weights} $

# aggregate by bus (occasionally needed) 

In [9]:
# all_actuals_df = all_actuals_df.groupby(lambda x: x.split("_")[0], axis=1).sum()
# all_scenarios_df = all_scenarios_df.groupby(lambda x: x.split("_")[0], axis=1).sum()
# all_actuals_df
# all_scenarios_df

<IPython.core.display.Javascript object>

# drop tz  (occasionally needed)

In [10]:
# # drop tz from actuals
# all_actuals_df.index = all_actuals_df.index.map(lambda t: t.replace(tzinfo=None))

# # drop tz from scenarios
# all_scenarios_df.index=all_scenarios_df.index.map(
#     lambda t: (t[0].replace(tzinfo=None), t[1], t[2].replace(tzinfo=None))
# )

# all_actuals_df
# all_scenarios_df



<IPython.core.display.Javascript object>

# save as .csv

In [11]:
# df = all_scenarios_df.copy()
# # if scenarios are single period, we can drop period_timestamp index level
# if n_periods == 1:
#     df.index=df.index.droplevel("period_timestamp")

# filename = './scenarios.csv'
# print("\nsaving all_scenarios_df to {}".format(filename))
# df
# df.to_csv(filename)

# # save weights
# filename = './weights.csv'
# print("\nsaving all_weights_df to {}".format(filename))
# all_weights_df
# all_weights_df.to_csv(filename)


# # take actuals corresponding to scenarios
# df=all_actuals_df
# df.index=df.index.rename("sim_timestamp")
# filename = './actuals.csv'
# print("\nsaving actuals to {}".format(filename))
# df
# df.to_csv(filename)


<IPython.core.display.Javascript object>

# saving as .aux 
will produce a separate .aux file for each actual and each senario period

In [12]:
# # save_dir = "/Users/isatkaus/projects/ecp/powerscenarios/data/scenarios-data/tamara/"
# save_dir = "./aux/"


# ###################################### save scenarios
# for i in range(len(all_scenarios_df)):

#     s = all_scenarios_df.iloc[i]

#     # create filename from multiindex, option 1: <simulation timestamp>_<scenario number>_<period timestamp>
#     # filename = str(s.name[0]).replace(' ','-')+'_'+str(s.name[1])+'_'+str(s.name[2]).replace(' ','-')

#     # create filename from multiindex, option 2: <simulation timestamp>_<scenario number>_<period number>
#     filename = (
#         str(s.name[0]).replace(" ", "-")
#         + "_S"
#         + str(s.name[1])
#         + "_P"
#         + str(((s.name[2] - s.name[0]).seconds // 60) // 5 + 1)
#         + ".aux"
#     )

#     filename = save_dir + filename

#     delimiter = " "
#     delimiter = "\t"

#     with open(filename, "w") as f:
#         # .aux header
#         _ = f.write("DATA (Gen, [BusNum,GenID,GenMW,GenStatus])\n")
#         _ = f.write("{\n")
#         # each series entry is one line
#         for k in range(len(s)):
#             _ = f.write(
#                 delimiter
#                 + s.index[k].split("_")[0]
#                 + delimiter
#                 + '"'
#                 + s.index[k].split("_")[2]
#                 + '"'
#                 + delimiter
#                 + str(s[k])
#                 + delimiter
#                 + '"Closed"'
#                 + "\n"
#             )
#         # .aux EOF
#         _ = f.write("}\n")


# ######################################## save actuals

# for i in range(len(all_actuals_df)):

#     s = all_actuals_df.iloc[i]

#     # create filename from multiindex, option 2: <simulation timestamp>_<scenario number>_<period number>
#     filename = str(s.name).replace(" ", "-") + "_A" + ".aux"

#     filename = save_dir + filename

#     delimiter = " "
#     delimiter = "\t"

#     with open(filename, "w") as f:
#         # .aux header
#         _ = f.write("DATA (Gen, [BusNum,GenID,GenMW,GenStatus])\n")
#         _ = f.write("{\n")
#         # each series entry is one line
#         for k in range(len(s)):
#             _ = f.write(
#                 delimiter
#                 + s.index[k].split("_")[0]
#                 + delimiter
#                 + '"'
#                 + s.index[k].split("_")[2]
#                 + '"'
#                 + delimiter
#                 + str(s[k])
#                 + delimiter
#                 + '"Closed"'
#                 + "\n"
#             )
#         # .aux EOF
#         _ = f.write("}\n")

<IPython.core.display.Javascript object>

# Scratch